# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [4]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [5]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [6]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [7]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [8]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [9]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


In [10]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [11]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [12]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [13]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [17]:
from itertools import chain

# создайте множество, чтобы проверять вхождения
vocab = set(chain.from_iterable([sent.split() for sent in corpus]))

In [18]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    if word in vocab:
        return 1
    else:
        return 0

In [22]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0 
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка 
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

## ваш код здесь
for bad_sent, true_sent in zip(bad, true):
    tokens = align_words(bad_sent, true_sent)
    for token_bad, token_good in tokens:
        if token_bad == token_good:
            y_true.append(0)
        else:
            y_true.append(1)
        y_pred.append(predict_mistaken(token_bad, vocab))

In [23]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.45      0.11      0.17      8707
          1       0.02      0.12      0.03      1305

avg / total       0.39      0.11      0.16     10012



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [24]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [25]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [26]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [27]:
P('апофеоз')

1.8025455548325345e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [28]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [29]:
%%time
correction('опефеоз')

CPU times: user 198 ms, sys: 6 ms, total: 204 ms
Wall time: 204 ms


'апофеоз'

Давайте поподробнее разберем, что происходит в функции edits.

In [30]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [31]:
splits[:10]

[('', 'опефеоз'),
 ('о', 'пефеоз'),
 ('оп', 'ефеоз'),
 ('опе', 'феоз'),
 ('опеф', 'еоз'),
 ('опефе', 'оз'),
 ('опефео', 'з'),
 ('опефеоз', '')]

In [32]:
deletes = [L + R[1:] for L, R in splits if R]

In [33]:
deletes[:10]

['пефеоз', 'оефеоз', 'опфеоз', 'опееоз', 'опефоз', 'опефез', 'опефео']

In [34]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [35]:
transposes[:10]

['поефеоз', 'оепфеоз', 'опфееоз', 'опеефоз', 'опефоез', 'опефезо']

In [36]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [37]:
len(replaces)

231

In [38]:
inserts = [L + c + R for L, R in splits for c in letters]

In [39]:
inserts[:10]

['йопефеоз',
 'цопефеоз',
 'уопефеоз',
 'копефеоз',
 'еопефеоз',
 'нопефеоз',
 'гопефеоз',
 'шопефеоз',
 'щопефеоз',
 'зопефеоз']

Для оценки используем просто долю правильных исправлений.

In [40]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.7333333333333333
10
0.8384615384615385
20
0.8441558441558441
30
0.8434343434343434
40
0.847953216374269
50
0.8518518518518519
60
0.8458333333333333
70
0.8502475247524752
80
0.8501594048884166
90
0.8526522593320236
100
0.8512544802867383
110
0.8547439126784215
120
0.8546017014694509
130
0.8534798534798534
140
0.8523581681476419
150
0.8580481622306717
160
0.8573113207547169
170
0.8568220101066817
180
0.8541666666666666
190
0.8552104208416834
200
0.8581048581048581
210
0.8593466424682396
220
0.8598971722365039
230
0.859958932238193
240
0.8639350599149594
250
0.8619170033051781
260
0.8628209637706648
270
0.8603786342123056
280
0.8606001936108422
290
0.8584758942457231
300
0.859375
310
0.8601173020527859
320
0.8605046781967678
330
0.8591549295774648
340
0.8582974137931034
350
0.8576288929599581
360
0.8575772934617334
370
0.8583478045150087
380
0.8595898673100121
390
0.8602558029369967
400
0.8619809788912085
410
0.8613615870153292
420
0.8611910560106265
430
0.8613861386138614
440
0.86255

In [41]:
print(correct/total)

0.8599680383539753


In [42]:
%%time
correction('нав')

CPU times: user 308 µs, sys: 1 µs, total: 309 µs
Wall time: 314 µs


'на'

In [43]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 3.43 s, sys: 59.6 ms, total: 3.49 s
Wall time: 3.65 s


'насмехатьсяаававттававаываываы'

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [44]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [45]:
# в питоне есть библиотеке для нахождения близких строк

In [46]:
from difflib import get_close_matches

In [47]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

CPU times: user 803 ms, sys: 11.8 ms, total: 815 ms
Wall time: 827 ms


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [49]:
import textdistance


In [50]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [51]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

CPU times: user 7.77 s, sys: 194 ms, total: 7.96 s
Wall time: 9.09 s


('апофеоз', 0.7142857142857143)

In [52]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

CPU times: user 1min 31s, sys: 1.97 s, total: 1min 33s
Wall time: 1min 50s


('апофеоз', 0.8571428571428572)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [53]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [54]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [55]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [56]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [57]:
%%time
get_closest_match_vec('опофеоз', X, vec)

CPU times: user 44.9 ms, sys: 14.3 ms, total: 59.3 ms
Wall time: 63.4 ms


['апофеоз', 'апофеозом', 'апофеоза']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [69]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    match_dict = {match: metric(text, match) \
        for match in get_closest_match_vec(text, X, vec, TOPN=10)}
    closest = sorted(match_dict, key=match_dict.get)[0]
    return closest

In [70]:
get_closest_hybrid_match('алкогнль', X, vec)

'алкоголь'

In [78]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# оцените качество также как и раньше (если будет долго работать возьмите кусок данных)
# посмотрите на ошибки
y_true = []
y_pred = []

for bad_sent, true_sent in zip(bad, true):
    tokens = align_words(bad_sent, true_sent)
    for token_bad, token_good in tokens:
        if token_bad != token_good:
            y_true.append(token_good)
            y_pred.append(get_closest_hybrid_match(token_bad, X, vec))

In [79]:
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(f"Precision: {precision_score(y_true, y_pred, average='micro')}")
print(f"Recall: {recall_score(y_true, y_pred, average='micro')}")
print(f"F1-score: {f1_score(y_true, y_pred, average='micro')}")

Accuracy: 0.4459770114942529
Precision: 0.4459770114942529
Recall: 0.4459770114942529
F1-score: 0.4459770114942529


In [80]:
for token_true, token_pred in zip(y_true, y_pred):
    if token_true != token_pred:
        print(f'Correct token: {token_true}, predicted token: {token_pred}')

Correct token: симпатичнейшее, predicted token: пластичнейшими
Correct token: получатся, predicted token: ополчатся
Correct token: очень, predicted token: очерчен
Correct token: насчет, predicted token: защищает
Correct token: в, predicted token: общем
Correct token: общем, predicted token: как
Correct token: как, predicted token: вы
Correct token: вы, predicted token: знаете
Correct token: знаете, predicted token: из
Correct token: из, predicted token: моего
Correct token: моего, predicted token: не
Correct token: недавнего, predicted token: давнего
Correct token: хорошо, predicted token: хорошее
Correct token: потому, predicted token: трампу
Correct token: что, predicted token: штат
Correct token: повтыкав, predicted token: фотка
Correct token: что-то, predicted token: что
Correct token: мощный, predicted token: мышкой
Correct token: хорошо, predicted token: хорошее
Correct token: молодежь, predicted token: молодежи
Correct token: сегодня, predicted token: сиднея
Correct token: навеш

Correct token: одним, predicted token: днем
Correct token: значит, predicted token: зачет
Correct token: пришлось, predicted token: пришлись
Correct token: сегодня, predicted token: сиднея
Correct token: когда, predicted token: ягод
Correct token: мясные, predicted token: мысе
Correct token: отдирала, predicted token: отдали
Correct token: отдыхаем, predicted token: отмечаемых
Correct token: конечно, predicted token: окрашенное
Correct token: ого-го, predicted token: ого
Correct token: вообще, predicted token: ваще
Correct token: рекомендуемо, predicted token: меморандуму
Correct token: равно, predicted token: ровно
Correct token: осталось, predicted token: остальном
Correct token: шкварчать, predicted token: ворчать
Correct token: слезятся, predicted token: связаться
Correct token: просмотреть, predicted token: просмотре
Correct token: что-то, predicted token: что
Correct token: как-то, predicted token: каток
Correct token: разве, predicted token: резать
Correct token: прожигам, predi

В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/